In [43]:
include("table.jl")

test (generic function with 1 method)

In [4]:
results = load_results()

Dict{Pair{String,String},Tuple{Float64,Float64,Float64,Float64,Dates.DateTime}} with 5 entries:
  "deep-EQ-d=10_k=1_gtype=… => (0.924901, 0.903212, 1.53125, 0.338883, DateTime…
  "flat-CNN-d=20_k=1_gtype… => (0.413285, 0.281718, 21.8228, 0.00534093, DateTi…
  "flat-CNN-d=10_k=1_gtype… => (0.431463, 0.312313, 10.4639, 0.00157763, DateTi…
  "flat-CNN-d=10_k=1_gtype… => (0.802241, 0.786567, 3.66699, 0.00159219, DateTi…
  "flat-CNN-d=20_k=1_gtype… => (0.774712, 0.759611, 8.76562, 0.00534856, DateTi…

In [11]:
parse_spec_string("d=15_k=1_gtype=ER_noise=Gaussian_mat=COR_raw")

(15, 1, "ER", "Gaussian", "COR", "Linear", true)

In [13]:
df = results2df(results)

,model,train_d,train_k,train_gtype,train_noise,train_mat,train_mec,test_d
,String,Int64?,Int64?,String?,String?,String?,String?,Int64
1,deep-EQ,10,1,SF,Gaussian,COR,Linear,10
2,flat-CNN,20,1,SF,Gaussian,COR,Linear,20
3,flat-CNN,10,1,SF,Gaussian,COR,Linear,10
4,flat-CNN,10,1,SF,Gaussian,COR,Linear,10
5,flat-CNN,20,1,SF,Gaussian,COR,Linear,20


In [41]:
# show all of df
show(df[1:3,:], allcols=true)

3×18 DataFrame
│ Row │ model    │ train_d │ train_k │ train_gtype │ train_noise │ train_mat │
│     │ String   │ Int64?  │ Int64?  │ String?     │ String?     │ String?   │
├─────┼──────────┼─────────┼─────────┼─────────────┼─────────────┼───────────┤
│ 1   │ deep-EQ  │ 10      │ 1       │ SF          │ Gaussian    │ COR       │
│ 2   │ flat-CNN │ 20      │ 1       │ SF          │ Gaussian    │ COR       │
│ 3   │ flat-CNN │ 10      │ 1       │ SF          │ Gaussian    │ COR       │

│ Row │ train_mec │ test_d │ test_k │ test_gtype │ test_noise │ test_mat │
│     │ String?   │ Int64  │ Int64  │ String     │ String     │ String   │
├─────┼───────────┼────────┼────────┼────────────┼────────────┼──────────┤
│ 1   │ Linear    │ 10     │ 1      │ SF         │ Gaussian   │ COR      │
│ 2   │ Linear    │ 20     │ 1      │ ER         │ Gaussian   │ COR      │
│ 3   │ Linear    │ 10     │ 1      │ ER         │ Gaussian   │ COR      │

│ Row │ test_mec │ raw  │ prec    │ recall  │ shd     │ tim

In [37]:
df[1:4,["model","train_d"]]

,model,train_d
,String,Int64?
1,deep-EQ,10
2,flat-CNN,20
3,flat-CNN,10
4,flat-CNN,10


In [ ]:
# sum(df.raw)

# I don't need train_k, because all of them are 1s
df = df[!, Not(All(:train_k, :train_noise))]

unique(df.train_mat)
unique(df.model)

In [28]:
function select_sep_df(df, model)
    sort(df[df.model .== model,
                    # test_d is same as train_d
                    Not(All(:train_k, :train_noise, :test_d))],
                 [:train_mec, :train_gtype, :test_gtype, :train_d, :test_k])
end

function select_ensemble_df(df, model)
    subdf = sort(df[df.model .== model, Not(All(:train_k, :train_noise, r"train_"))],
                 [:test_gtype, :test_d, :test_k])
    show(subdf, allrows=true)
    nothing
end

select_sep_df (generic function with 1 method)

In [ ]:
# 1. all the models in separate mode
select_sep_df(df, "deep-FC")

In [30]:
select_sep_df(df, "deep-EQ")

,model,train_d,train_gtype,train_mat,train_mec,test_k,test_gtype,test_noise
,String,Int64?,String?,String?,String?,Int64,String,String
1,deep-EQ,10,SF,COR,Linear,1,SF,Gaussian


In [44]:
table_cmp_baseline(df)

,model,train_d,prec,recall,shd
,String,Int64?,Float32,Float32,Float32
1,deep-EQ,10,92.5,90.3,1.5
2,flat-CNN,10,80.2,78.7,3.7
3,flat-CNN,20,77.5,76.0,8.8


In [45]:
table_ER24(df)

,model,train_d,train_gtype,test_gtype,raw,prec,recall,shd,time
,String,Int64?,String?,String,Bool,Float32,Float32,Float32,Float32
1,deep-EQ,10,SF,SF,0,92.5,90.3,1.5,0.3


In [46]:
table_raw(df)

,model,train_d,prec,recall,shd,time,test_d
,String,Int64?,Float32,Float32,Float32,Float32,Int64
1,deep-EQ,10,92.5,90.3,1.5,0.3,10
2,flat-CNN,10,43.1,31.2,10.5,0.0,10
3,flat-CNN,10,80.2,78.7,3.7,0.0,10
4,flat-CNN,20,41.3,28.2,21.8,0.0,20
5,flat-CNN,20,77.5,76.0,8.8,0.0,20


In [47]:
table_transfer_k(df)

,model,train_d,train_gtype,test_k,test_gtype,raw,prec,recall,shd
,String,Int64?,String?,Int64,String,Bool,Float32,Float32,Float32
1,deep-EQ,10,SF,1,SF,0,92.5,90.3,1.5


In [48]:
table_transfer_noise(df)

,model,train_d,train_gtype,train_noise,test_noise,raw,prec,recall
,String,Int64?,String?,String?,String,Bool,Float32,Float32
1,deep-EQ,10,SF,Gaussian,Gaussian,0,92.5,90.3


In [49]:
table_transfer_gtype(df)

,model,train_d,train_gtype,test_gtype,raw,prec,recall,shd,time
,String,Int64?,String?,String,Bool,Float32,Float32,Float32,Float32


In [50]:
table_ensemble(df)

,model,test_d,test_gtype,raw,prec,recall,shd,time
,String,Int64,String,Bool,Float32,Float32,Float32,Float32
